In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import cftime
import datetime
import xarray as xr
import wget
import os
import glob
import matplotlib.pyplot as plt
import dask
%matplotlib inline

In [4]:
os.chdir('/Users/caramelo/Documents/0000_Unu_Engineering/wasabi_cloud_storage/gem/TMP')
grib_list=glob.glob('*.grib2')
grib_list

['CMC_reg_TMP_TGL_2_ps10km_2021030112_P083.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P006.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P043.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P022.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P067.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P058.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P020.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P065.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P004.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P041.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P081.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P039.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P000.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P045.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P078.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P019.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P024.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P061.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021030112_P026.grib2',
 'CMC_reg_TMP_TGL_2_ps10km_2021

In [5]:
ds=xr.open_mfdataset(grib_list,concat_dim='valid_time',engine='cfgrib',combine='nested')

In [6]:
# Convert to celsius
ds=ds-273.15 #passage en celsius
t2m = ds.t2m 

# copy attributes to get nice figure labels and change Kelvin to Celsius
t2m.attrs = ds.t2m.attrs

t2m.attrs["units"] = "deg C"

In [9]:
ds

<xarray.Dataset>
Dimensions:            (valid_time: 85, x: 935, y: 824)
Coordinates:
    time               datetime64[ns] 2021-03-01T12:00:00
    step               (valid_time) timedelta64[ns] 3 days 11:00:00 ... 1 day...
    heightAboveGround  int64 2
    latitude           (y, x) float64 dask.array<chunksize=(824, 935), meta=np.ndarray>
    longitude          (y, x) float64 dask.array<chunksize=(824, 935), meta=np.ndarray>
  * valid_time         (valid_time) datetime64[ns] 2021-03-04T23:00:00 ... 20...
Dimensions without coordinates: x, y
Data variables:
    t2m                (valid_time, y, x) float32 dask.array<chunksize=(1, 824, 935), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal 
    history:                 2021-03-01T12:46:57 GRIB to CDM+CF via cfgrib-0....

In [7]:
min_lon=-90
min_lat=40
max_lon=-55
max_lat=75

mask_lon= (ds.longitude >=min_lon) & (ds.longitude <=max_lon)
mask_lat= (ds.latitude >=min_lat) & (ds.latitude <=max_lat)

cropped_ds = ds.where(mask_lon & mask_lat, drop=True)
cropped_ds

In [10]:
fig,ax=plt.subplots(figsize=(15,15))

countries=gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ds.t2m.isel(valid_time=1).plot(ax=ax)
countries.plot(ax=ax,facecolor='none',edgecolor='red')

IndexError: index -1 is out of bounds for axis 0 with size 0